## SET THE MARKET

In [1]:
MARKET = "NSE"

## IMPORTS, CONNECTIONS, LOG, TIMER

In [2]:
import asyncio
import pickle
import sys
import time
import pandas as pd
import random
import copy

from collections import defaultdict
from datetime import datetime
from pprint import pprint

from ib_insync import *

from ib01_getsyms import get_syms
from support import timestr

from typing import Callable, Coroutine

random.seed(8888)

if sys.version_info[0] == 3 and sys.version_info[1] >= 8 and sys.platform.startswith('win'):
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

In [3]:
import nest_asyncio
util.startLoop()
nest_asyncio.apply()

pd.options.display.max_columns = None

In [7]:
HOST = '127.0.0.1'
PORT = 4004 if MARKET.upper() == 'NSE' else 4002 # Paper trades!
CID = 0
MASTERCID = 10

ib = IB()
FSPATH: str='./data/'

In [5]:
# Direct logs to file with level at WARNING (30)
util.logToFile(path='./data/data.log', level=30)
with open('./data/data.log', 'w'): # to clear the log
    pass

# Making df_opts

In [ ]:
df_chains = pd.read_pickle(FSPATH+'df_chains.pkl')

df_ch1 = pd.concat([df_chains.assign(right='P'), 
          df_chains.assign(right='C')], 
          ignore_index=False)

puts = [Option(symbol=s, lastTradeDateOrContractMonth=e, strike=k, right='P', exchange=x) 
                    for s, e, k, x in 
                    zip(df_ch1.symbol, df_ch1.expiry, df_ch1.strike, ['NSE' 
                        if MARKET.upper() == 'NSE' else 'SMART']*len(df_ch1))]

calls = [Option(symbol=s, lastTradeDateOrContractMonth=e, strike=k, right='C', exchange=x) 
                    for s, e, k, x in 
                    zip(df_ch1.symbol, df_ch1.expiry, df_ch1.strike, ['NSE' 
                        if MARKET.upper() == 'NSE' else 'SMART']*len(df_ch1))]

In [9]:
pd.read_pickle(FSPATH+'df_nsesyms.pkl')

,ctype,symbol,expiryM,lot,exchange,currency
0,Index,NIFTY50,2020-08,75,NSE,INR
1,Index,BANKNIFTY,2020-08,25,NSE,INR
2,Index,BANKNIFTY,2020-10,25,NSE,INR
3,Index,NIFTY50,2020-10,75,NSE,INR
4,Index,NIFTY50,2020-09,75,NSE,INR
...,...,...,...,...,...,...
412,Stock,SAIL,2020-08,19000,NSE,INR
413,Stock,RECLTD,2020-08,6000,NSE,INR
414,Stock,BAJFINANC,2020-08,250,NSE,INR
415,Stock,AMBUJACEM,2020-09,3000,NSE,INR


## Qualifying options

In [ ]:
async def qualCoro(cts):
    contracts = await ib.qualifyContractsAsync(cts)
    await asyncio.sleep(20)
    return contracts

In [ ]:
raw_opts = copy.copy(puts) + copy.copy(calls)
cts = random.sample(raw_opts, len(raw_opts))[:100000]
todo = set()
result = set()
pkl_timeout = 10
total_timeout = 0

In [ ]:
# cts[:25]

In [ ]:
async def qualTasks(cts: list) -> None:
    blk=1000
    optblks = [cts[i: i+blk] for i in range(0, len(cts), blk)]

    for b in optblks:
        todo.add(asyncio.create_task(ib.qualifyContractsAsync(*b), 
                                 name=f'{b[0].symbol}{b[0].lastTradeDateOrContractMonth[:-2]}{b[0].right}{b[0].strike}:'+\
                                             f'{b[len(b)-1].symbol}{b[len(b)-1].lastTradeDateOrContractMonth[:-2]}{b[len(b)-1].right}{b[len(b)-1].strike}'))


    start = time.time()
    
    while len(todo):
        
        done, pending = await asyncio.wait(todo, timeout=pkl_timeout)
        
        # remove done task from todo after the timeout, update result and pickle it
        todo.difference_update(done)
        result.update(done)
        
        # report pendings
        pending_names = (t.get_name() for t in todo)
        print(f"{len(todo)}: "+ " ".join(sorted(pending_names))[-75:])
        
        # check for total_timeout
        if total_timeout > 0.0:
            if time.time() - start > total_timeout:
                print(f'\nProgram exceeded total_timeout of {total_timeout} seconds')
                print(f'Cancelling pending todos')
                for task in todo:
                    task.cancel()
                done, pending = await asyncio.wait(todo, timeout=1.0)
                todo.difference_update(done)
                todo.difference_update(pending)
                
    end = time.time()
    print(f"Took {int(end-start)} seconds")
        
#     for c in cts:
#         todo.add(asyncio.create_task(qualCoro(c)))

In [ ]:
%%time
with ib.connect(HOST, PORT, CID) as ib:
    ib.run(qualTasks(cts))

In [ ]:
len([i for r in result for i in r.result()])

In [ ]:
for task in todo:
    task.cancel()

In [ ]:
len(cts)

In [ ]:
async def progressQuals(cts,
                       algo: Callable[..., Coroutine],
                       pkl_timeout: float=2.0,
                       total_timeout: float=0.0,
                       FSPATH: str='./data/',
                       ) -> None:

        # create a task for the algo
    task = asyncio.create_task(algo(cts), name=algo.__name__)
    
    todo.add(task) # add task to the asyncio loop
    
    start = time.time()
    
    while len(todo):
        
        done, pending = await asyncio.wait(todo, timeout=pkl_timeout)
        
        # remove done task from todo after the timeout, update result and pickle it
        todo.difference_update(done)
        result.update(done)
        
        # report pendings
        pending_names = (t.get_name() for t in todo)
        print(f"{len(todo)}: "+ " ".join(sorted(pending_names))[-75:])
        
        # check for total_timeout
        if total_timeout > 0.0:
            if time.time() - start > total_timeout:
                print(f'\nProgram exceeded total_timeout of {total_timeout} seconds')
                print(f'Cancelling pending todos')
                for task in todo:
                    task.cancel()
                done, pending = await asyncio.wait(todo, timeout=1.0)
                todo.difference_update(done)
                todo.difference_update(pending)
                
    end = time.time()
    print(f"Took {int(end-start)} seconds")

In [ ]:
with ib.connect(HOST, PORT, CID) as ib:
    ib.run(progressQuals(cts=cts, algo=qualTasks))

In [ ]:
result